<a href="https://colab.research.google.com/github/Bruno-Noriaki/Pokemon_ML/blob/main/ML_Pokemon_Battle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import tree
import pydot
from sklearn.tree import export_graphviz
import ipywidgets as widgets

In [2]:
pokemon = pd.read_csv('/content/pokemon.csv')
combat = pd.read_csv('/content/combats.csv')
test = pd.read_csv('/content/tests.csv')

O Banco de dados utilizado está disponível em https://www.kaggle.com/datasets/terminus7/pokemon-challenge

# Descrição

Nesse pequeno projeto, o objetivo é utilizar os principais status de batalha a fim de prever o vencedor de um combate entre dois pokémons

# Análise do Data Set de pokemon

Há apenas um dado com marcação incorreta em nome e os dados vazios em Type 2 são apenas de pokemons que não possuem tipo secundário

In [3]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        799 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


In [4]:
pokemon[pokemon['Name'].isnull()]
#Dado faltante, um preechimento incorreto

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
62,63,NaN,Fighting,NaN,65,105,60,60,70,95,1,False


In [5]:
a = list(range(58, 65, 1))

pokemon[58:65]
#O nome em falta trata-se de Primeape

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
58,59,Persian,Normal,NaN,65,70,60,65,65,115,1,False
59,60,Psyduck,Water,NaN,50,52,48,65,50,55,1,False
60,61,Golduck,Water,NaN,80,82,78,95,80,85,1,False
61,62,Mankey,Fighting,NaN,40,80,35,35,45,70,1,False
62,63,NaN,Fighting,NaN,65,105,60,60,70,95,1,False
63,64,Growlithe,Fire,NaN,55,70,45,70,50,60,1,False
64,65,Arcanine,Fire,NaN,90,110,80,100,80,95,1,False


In [6]:
pokemon['Name'] = pokemon['Name'].fillna('Primeape')

In [7]:
#Para os dados vazio de type_2, será preechido com 0
pokemon['Type 2'] = pokemon['Type 2'].fillna(0)

In [8]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      800 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


## Tratamento de dados Categóricos

O número será apenas utilizado como marcador do pokemon, mas será necessário alterar para dummies os tipos primário e secundário. Dummy marcado com 1 para o tipo primário e 2 para secundário

In [9]:
def definicao_tipo(lista):
  types_unique = pokemon['Type 1'].unique()
  classification = []
  for i in range(0,len(lista)):
    tipo = []
    for tipagem in types_unique:
      if pokemon['Type 1'][i] == tipagem:
        tipo.append(1)
      elif pokemon['Type 2'][i] == tipagem:
        tipo.append(2)
      else:
        tipo.append(0)
    classification.append(tipo)
  return classification

In [10]:
tipos = definicao_tipo(pokemon)

In [11]:
df_types = pd.DataFrame(tipos, columns = pokemon['Type 1'].unique())

In [12]:
df_final = pd.merge(pokemon, df_types, left_index=True, right_index=True)

In [13]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      800 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
 12  Grass       800 non-null    int64 
 13  Fire        800 non-null    int64 
 14  Water       800 non-null    int64 
 15  Bug         800 non-null    int64 
 16  Normal      800 non-null    int64 
 17  Poison      800 non-null    int64 
 18  Electric    800 non-null    int64 
 19  Ground      800 non-null    int64 
 20  Fairy     

In [14]:
#drop de colunas iniciais relacionadas aos tipos
pokemon_f = df_final.drop(['Type 1','Type 2'], axis = 1)

In [15]:
#Transformarei a coluna de numeração de pokemon
#como uma categórica a fim de evitar qualquer viés equivocado
pokemon_f['#'] = pokemon_f['#'].apply(lambda x: str(x))

In [16]:
#Por fim, mudarei legendary de booleano para binário
pokemon_f['Legendary'] = pokemon_f['Legendary'].replace({True:1,False:0})

# Dados de combates e teste


In [17]:
combat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   First_pokemon   50000 non-null  int64
 1   Second_pokemon  50000 non-null  int64
 2   Winner          50000 non-null  int64
dtypes: int64(3)
memory usage: 1.1 MB


## Transformação dos dados em categóricos para evitar viéses

In [18]:
combat[['First_pokemon','Second_pokemon','Winner']] = combat[['First_pokemon','Second_pokemon','Winner']].astype(str)
test[['First_pokemon','Second_pokemon']] = test[['First_pokemon','Second_pokemon']].astype(str)

Aqui converterei a coluna de winner para um binário indicando o vencedor, para evitar erros onde num combate entre dois pokemon, haja um terceiro vencedor

In [19]:
for i in range(0, len(combat)):
  if combat['Winner'][i] == combat['First_pokemon'][i]:
    combat['Winner'][i] = '0'
  else:
    combat['Winner'][i] = '1'

In [20]:
combat.head()

,First_pokemon,Second_pokemon,Winner
0,266,298,1
1,702,701,1
2,191,668,1
3,237,683,1
4,151,231,0


## União de DataFrames

In [21]:
combat_m = combat.merge(pokemon_f, how='left', left_on = 'First_pokemon', right_on ='#')
test_1 = test.merge(pokemon_f, how='left', left_on = 'First_pokemon', right_on ='#')

In [22]:
combat_m = combat_m.merge(pokemon_f, how='left', left_on = 'Second_pokemon', right_on ='#')
test_1 = test_1.merge(pokemon_f, how='left', left_on = 'Second_pokemon', right_on ='#')

# Treino e teste com base de combat

Para esse treinamento e previsão, será utilizado o modelo de árvore de decisões, com base na utilização de muitas variáveis categóricas

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [24]:
X = combat_m.drop(['Winner','#_x','Name_x','#_y','Name_y','First_pokemon','Second_pokemon'],axis = 1)
Y = combat_m['Winner']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2)

In [25]:
from sklearn.tree import DecisionTreeClassifier

tree_reg = DecisionTreeClassifier()
tree_reg.fit(x_train, y_train)

DecisionTreeClassifier()

In [26]:
y_pred = tree_reg.predict(x_test)
combat_m['Pred'] = tree_reg.predict(X)

In [27]:
result = []
for i in range(0,len(combat_m)):
  if combat_m['Winner'][i] == combat_m['Pred'][i]:
    result.append(0)
  else:
    result.append(1)

In [28]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average = 'macro')
f1 = f1_score(y_test, y_pred, average = 'macro')
recall = recall_score(y_test, y_pred, average = 'macro')

In [29]:
accuracy

0.946

In [30]:
precision

0.9458431599043815

In [31]:
f1

0.9458524566078634

In [32]:
recall

0.9458618295771799

Por se tratar de dados preditivos categóricos, utilizei as medições acima a fim de observar a qualidade do modelo, todas os parâmetros apresentaram ótimo resultado, até por se tratar de uma base pequena e com variáveis bem definidas e lógicas de combate claras

# Geração dos resultados de batalha do DataFrame de Teste

In [33]:
x_df = test_1.drop(['#_x','Name_x','#_y','Name_y','First_pokemon','Second_pokemon'], axis = 1)

In [34]:
y_winner = tree_reg.predict(x_df)
x_df['Winner'] = y_winner

## Gerando PNG da árvore de decisão

In [35]:
features = x_df.drop('Winner', axis = 1 ).columns

In [36]:
tree.dot = tree.export_graphviz(tree_reg, out_file = 'tree_pokemon', feature_names = features,
                     class_names = x_df['Winner'], filled = True, rounded = True)

In [37]:
graph = pydot.graph_from_dot_file('tree_pokemon')
output_file = 'graph.png'
graph[0].write_png(output_file)

## Análise das importâncias das variáveis

In [38]:
importances = tree_reg.feature_importances_
features = x_df.drop('Winner', axis = 1).columns
imp = pd.DataFrame({'Feature': features, 'Importance': importances})

In [39]:
imp.sort_values(by = 'Importance', ascending = False).head(10)

,Feature,Importance
31,Speed_y,0.406629
5,Speed_x,0.391803
27,Attack_y,0.022843
1,Attack_x,0.021363
46,Ghost_y,0.012917
0,HP_x,0.009290
20,Ghost_x,0.007463
26,HP_y,0.007353
12,Normal_x,0.007350
28,Defense_y,0.007286


Nota-se uma grande influência do status de Speed e logo em seguida de Ataque, pórém bem com influência bem menor, os demais status permanecem com influência bastante baixa.

# Pequena API para geração de Combates

In [40]:
#Função para retornar os status de cada pokemon no formato aceitável dentro do meu modelo

def Battle():
  Nomes_pokemon = pokemon_f['Name'].unique()

  print('Lista de Pokémons Disponíveis')
  dropdown = widgets.Dropdown(options=Nomes_pokemon,
                                disabled=False)
  display(dropdown)


  first = input(str('Digite o nome do primeiro pokemon: ')).title().strip()
  second = input(str('Digite o nome do segundo pokemon: ')).title().strip()

  stat_1 = pokemon_f[pokemon_f['Name'] == first].drop(['Name','#'], axis = 1)
  stat_2 = pokemon_f[pokemon_f['Name'] == second].drop(['Name','#'], axis = 1)
  stat_total = []

  for  _, value in stat_1.items():
    stat_total.append(value.values[0])

  for _, value in stat_2.items():
    stat_total.append(value.values[0])

  stat_total = pd.DataFrame(stat_total)
  stat_total = stat_total.transpose()

  vencedor = tree_reg.predict(stat_total);

  if vencedor == '0':
    vencedor = first
  else:
    vencedor = second
  print('---'*20)
  print(f'O vencedor dessa batalha foi {vencedor.upper()}')
  print('---'*20)

# API

In [41]:
Battle();

Lista de Pokémons Disponíveis


Dropdown(options=('Bulbasaur', 'Ivysaur', 'Venusaur', 'Mega Venusaur', 'Charmander', 'Charmeleon', 'Charizard'…

Digite o nome do primeiro pokemon: Pikachu
Digite o nome do segundo pokemon: Entei
------------------------------------------------------------
O vencedor dessa batalha foi ENTEI
------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
